# CNN Baseline Model

## Load Data

In [58]:
import numpy as np
import os
from PIL import Image

### Small Dataset Processing

In [59]:
def data_processing():

    path = os.path.abspath("toydata")

    tiff_files = []

    for f in os.listdir(path):
        if f.endswith('.tif'):
            tiff_files.append(path + '/' + f)

    tiff_dict = {}

    # dictionary of tiff files
    for f in tiff_files:
        k = f.split('/')[-1].split('.tif')[0]
        tiff_dict[k] = f

    # convert to np array
    fire_id = Image.open(tiff_dict['fireid'])
    fire_id = np.array(fire_id)
    fire_id[fire_id == -9999] = 0

    fireline = Image.open(tiff_dict['Global_fire_atlas_firelinecrop'])
    fireline = np.array(fireline)
    fireline[fireline == -9999] = 0

    # get list of unique fire_ids
    fire_ids = set()

    for row in fire_id:
        for val in row:
            fire_ids.add(val)

    # remove 0 from fire_ids set because it does not denote a fire
    fire_ids.remove(0)

    # get dict with key value pairs of fire_id and an empty dict
    fire_data_dict = {}

    for id in fire_ids:
        id = str(id)
        fire_data_dict[id] = {}

    for id in fire_ids:
        indices = np.where(fire_id == id, 1, 0)
        fire_data_dict[str(id)] = indices
        
    return fire_data_dict, fireline

In [60]:
fire_data_dict, fireline = data_processing()

### Toy Dataset

In [61]:
# create a toy dataset to test create_one_matrices

toy_fire_data_dict = {}
toy_fire_data_dict['1'] = np.mat('0 0 0 0 0; 0 1 1 1 0; 0 1 1 0 0; 0 0 1 1 0; 0 1 1 1 1; 0 0 0 1 1')
toy_fireline = np.mat('0 0 0 0 0; 0 2 2 2 0; 0 2 2 0 0; 0 0 3 3 0; 0 3 3 3 3; 0 0 0 4 4')

### Create One-Hot Fire-Day Matrices

In [62]:
def create_one_hot_matrices(data_dict, fireline):
    '''
    This returns a dictionary with the structure:
    {
        "fire_id": {
            "day_of_year": one-hot encoded 2D array of fire spread on that day,
            "day_of_year": one-hot encoded 2D array of fire spread on that day
        }

    }
    '''
    
    fire_data_dict = {}
    
    for key, val in data_dict.items():
        data = {}
        for index, x in np.ndenumerate(val):
            for y in range(1, 366):
                indices = np.where(
                    ((fireline == y) & (val[index] == 1)),
                    1,
                    0
                )

                if np.sum(indices) > 0:
                    data[str(y)] = indices
                    
        fire_data_dict[str(key)] = data
        
    return fire_data_dict

In [63]:
# Test on toy dataset
toy_fire_data_dict = create_one_hot_matrices(toy_fire_data_dict, toy_fireline)
print(toy_fire_data_dict)

{'1': {'2': array([[0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]]), '3': array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0],
       [0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0]]), '4': array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1]])}}


In [66]:
def create_supervised_data(fire_data_dict):
    '''
    Turn this dictionary:
    {
        "fire_id": {
            "day_of_year": one-hot encoded 2D array of fire spread on that day,
            "day_of_year": one-hot encoded 2D array of fire spread on that day
        }

    }
    
    into:
    [
        (one-hot encoded 2D array of fire spread on that day_1, one-hot encoded 2D array of fire spread on day_2),
        (one-hot encoded 2D array of fire spread on that day_2, one-hot encoded 2D array of fire spread on day_3),
    ]
    '''
    
    train_labels = []

    for key, value in fire_data_dict.items():
        burn_matrices = list(value.values())
        
        for index, day in enumerate(burn_matrices):

            if index < len(burn_matrices) - 1:
                day_1 = burn_matrices[index]
                day_2_index = index + 1
                day_2 = burn_matrices[day_2_index]
                
                pair = (day_1, day_2)
                train_labels.append(pair)

    return train_labels

x = create_supervised_data(toy_fire_data_dict)

[(array([[0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]]), array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0],
       [0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0]])), (array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0],
       [0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0]]), array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1]]))]


In [ ]:
# Run on small dataset
small_fire_data_dict = create_one_hot_matrices(fire_data_dict, fireline)
small_dataset = create_supervised_data(small_fire_data_dict)